# Grades Coorelation between Courses Prerequisits
this notebook shows the coorelation of grades between all courses and their prerequisits in a given curriculum. The input of the script is the curriculum of the academic program and students grades that have/are pursing the same program.

In [2]:
using CSV
using DataFrames
#using Dates
using Plots
#using PlotlyJS
using Statistics
using DataFramesMeta
#using Distributions
using StatsBase
using Interact
using Blink
using StatsPlots
using HTTP
#using Query
#using FreqTables
using Statistics
using CurricularAnalytics

WebIO._IJuliaInit()

In [3]:
#student data
stdata= CSV.read("Grades.csv",DataFrame);

#curriculum
curr = read_csv("ua_chemeng.csv");

In [4]:
# verify the input curriculm is valid
isvalid_curriculum(curr)

true

In [5]:
#create a dataframe (df) that join the student_data to itself for easier indexing between courses prerequistis
df=innerjoin(stdata, stdata, on=:EMPLID, makeunique=true)
head(df)

,EMPLID,COURSE,COURSENAME,GRADE,SCORE,COURSE_1,COURSENAME_1
,Int64,String,String,String,Int64,String,String
1,18929,RELI481A,Hist Of E Asian Buddhism,A,4,RELI481A,Hist Of E Asian Buddhism
2,29812,LING515,Phonological Phonetics,A,4,LING515,Phonological Phonetics
3,29812,LING515,Phonological Phonetics,A,4,ENGL557B,Contemp British Lit
4,29812,LING515,Phonological Phonetics,A,4,LING503,Found Syntactic Theory I
5,29812,LING515,Phonological Phonetics,A,4,LING573,Natural Lang Processing
6,29812,LING515,Phonological Phonetics,A,4,LING501,Formal Found Linguistics


In [6]:
#record all courses (that have prerequisit(s)) and its prerequisit(es) in a dataframe courses
courses = DataFrame(Prerequisite=String[],Course=String[])
for i in 1:curr.num_courses
    #temp=c.courses[i].requisites
    pre_list=collect(keys(curr.courses[i].requisites))
    if !isempty(pre_list)
        for j in 1:size(pre_list)[1]
            push!(courses,[string(curr.courses[pre_list[j]].prefix,"",curr.courses[pre_list[j]].num) string(curr.courses[i].prefix,"",curr.courses[i].num)])
        end
    end
end

In [7]:
head(courses)

,Prerequisite,Course
,String,String
1,MATH122A,MATH122B
2,MATH122B,MATH129
3,MATH122B,CHEM152
4,ENGL101,ENGL102
5,MATH122B,PHYS141
6,MATH122B,AME105


In [8]:
#create vectors of grade for each course (x) and its prerequisites in (y) 
x=Vector{Vector{Int64}}(undef,size(courses)[1]);
y=Vector{Vector{Int64}}(undef,size(courses)[1]);
for i in 1:size(courses)[1]
    course = courses[i,"Course"]
    pre_course = courses[i,"Prerequisite"]
    temp_df=filter(row -> row.COURSE == pre_course && row.COURSE_1 == course ,df)
    x[i] = temp_df.SCORE
    y[i] = temp_df.SCORE_1
end   

In [8]:
#is there any empty vectors for any courses ?
for i in 1:size(courses)[1]
   if isempty(x[i])
        @show i
    end
end

In [9]:
#create dataframe (curr) and calculate the statistical coorelation between x and y
corr = DataFrame(Prerequisite=String[],Course=String[],Correlation=Float64[])
for i in 1:size(courses)[1]
    push!(corr,[courses[i,2] courses[i,1] Statistics.cor(x[i],y[i])])
end
corr

,Prerequisite,Course,Correlation
,String,String,Float64
1,MATH122B,MATH122A,0.367693
2,MATH129,MATH122B,0.458953
3,CHEM152,MATH122B,0.413517
4,ENGL102,ENGL101,0.432592
5,PHYS141,MATH122B,0.383004
6,AME105,MATH122B,NaN
7,CHEM243A,CHEM152,0.583673
8,AME205,AME105,0.583221
9,CHEE201L,AME105,-0.365148
